In [ ]:
import faiss
import numpy as np
from openai import OpenAI
import os

client = OpenAI(api_key="Api key")  

def load_faiss_index(index_path: str):
    """Load a FAISS index from the specified file path."""
    return faiss.read_index(index_path)

def load_text_chunks(path: str):
    """Load text chunks (retrieved documents) from a file."""
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

def generate_query_embedding(query: str):
    """Generate an embedding vector for the input query using OpenAI."""
    response = client.embeddings.create(
        input=query,
        model="text-embedding-ada-002"
    )
    return np.array(response.data[0].embedding).astype("float32").reshape(1, -1)

def search_index(faiss_index, query_embedding, k: int = 5):
    """Search the FAISS index and return top-k indices for similar chunks."""
    distances, indices = faiss_index.search(query_embedding, k)
    return indices[0]

def build_prompt(query: str, relevant_chunks: list):
    """Construct the full prompt for GPT using context and the user's query."""
    context = "\n".join(relevant_chunks)
    
    return f"""Context:
{context}

Query:
{query}

Answer:"""

def generate_response(query: str, relevant_chunks: list, model: str = "gpt-4-turbo", max_tokens: int = 500):
    """Generate a response from the LLM using the query and context chunks."""
    
    if not relevant_chunks:
        return "I'm sorry, but I couldn't find enough relevant medical information based on your symptoms. We will check back with the doctor"

    system_prompt = (
        "You are a helpful and knowledgeable medical assistant. "
        "The user will provide medical symptoms, and your task is to suggest possible diagnoses or causes based solely on the information provided and the retrieved documents.\n\n"
        "Guidelines:\n"
        "- Always use only the information from the context provided.\n"
        "- If no relevant information is found, clearly state that the context doesn’t contain sufficient details.\n"
        "- Never make up diagnoses, symptoms, or treatments.\n"
        "- Be cautious and avoid medical certainty.\n"
        "- Use clear, concise, and professional language.\n"
        "- Do not assume or guess beyond the provided context.\n"
        "- This is a medical system, do not tell the user to consult a healthcare professional"
        "If the user gives something very abstract ask it to redescribe it, "
        "If the q"
    )

    user_prompt = build_prompt(query, relevant_chunks)

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

def answer_query(query: str, index_path: str, chunks_path: str, k: int = 5):
    """Full RAG pipeline: From query to response."""
    faiss_index = load_faiss_index(index_path)
    chunks = load_text_chunks(chunks_path)
    query_embedding = generate_query_embedding(query)
    indices = search_index(faiss_index, query_embedding, k)
    relevant_chunks = [chunks[i] for i in indices if i < len(chunks)]
    return generate_response(query, relevant_chunks)

# === Example ===


# Get the directory of the current script
script_dir = os.path.dirname(os.path.abspath(__file__))


index_path = os.path.join(script_dir, "index")
chunks_path = os.path.join(script_dir, "chunks.txt")

query = "I feel sick and weird "

response = answer_query(query, index_path, chunks_path, k=5)
print("Answer:", response)
